In [ ]:
% matplotlib inline

import numpy as np
import numpy.ma as ma
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.basemap import Basemap, cm
import seawater as sw

import MITgcmutils as mit
from MITgcmutils import cs

In [ ]:
# script to interpolate ECCO2 output onto ECCO-Godae grid

rmwfile='/Users/sclayton/Documents/MIT_work/comparison/hrmp_e2_cs510_to_eg_v3_noland.nc'

# ECCO-Godae maskfile and number of vertical levels (needed to read mask).
# outhFacCfile='hFacC_eg.data';
outhFacCfile='/scratch/stephd/DARWIN12/run89d/Year10_month/grid.*.nc'
nzout=23

# Load weights information
# NEED NETCDF HERE #####################################################################
nc=netcdf(rmwfile,'r');
tmp=nc('num_links'); nlinks=tmp(:);
tmp=nc('src_grid_size'); ndin=tmp(:);
ndxout=nc{'dst_grid_dims'}(1);
ndyout=nc{'dst_grid_dims'}(2);
rmp_wgts=nc{'remap_matrix'}(:,:);
rmp_src_i=nc{'src_address'}(:);
rmp_dst_i=nc{'dst_address'}(:);
close(nc);

hFacC = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/hFacC')

outhfacc = np.fromfile(in_file, dtype = '>f')
outhfacc = outhfacc.reshape(23, 160, 360)

dstmask1 = outhfacc[0,:,:]
dstmask1[dstmask1!=0]=1

# Create the weights matrix
###### NEED TO CONVERT THIS #########################
rm_mat=sparse(rmp_dst_i(:),rmp_src_i(:),rmp_wgts(:,1),ndxout*ndyout,ndin);

# Regrid data

infile=sprintf('/scratch/jahn/ecco2/cube84/assembled/L1/ETAN/ETAN_day.%010d.data',it)
phiall=np.fromfile(infile).reshape(510, 3060)

fooall=np.zeros(160,360)

phi=phiall
phi[phi==0]=1e-30

phi[hFacC==0]=0
phi = phi.reshape(1,510,6,510)
phi = np.squeeze(phi);
phi = np.transpose(phi,(0, 2, 1))
    
# Create a field with 1 over water and 0 over land
phi_one=phi(:).*0;
phi_one(find(phi(:)~=0))=1;
    
# Bin average the src field to the dst mesh
foo = rm_mat*phi.flatten(1)
foo = full(foo)
foo = foo.reshape(ndyout, ndxout)

# Bin average the land-sea mask field to the dst mesh
foo_one = rm_mat*phi_one
foo_one = full(foo_one); # LOOK UP FULL FUNCTION
foo_one = foo_one.reshape(ndyout, ndxout)
a = foo_one[foo_one!=0]
foo_one[foo_one!=0]=1./a
    
# Zap points for which less than 1/alpha of the cell is wet.
alpha = 20
foo_one[foo_one>alpha] = 0.
foo = foo*foo_one
foo = foo *dstmask1
foo[dstmask1==0] = 0.
fooall = foo
fooall[isnan(fooall)]==0 # CHECK THIS LINE!!!

#  Write out the fooall field


